Importing Required Libraries

In [1]:

import os
import shutil
import random

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Creating Paths for Training and Validation

In [2]:

data_dir = 'CEDAR'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

In [3]:
train_dir = 'Data/train'
val_dir = 'Data/val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create subdirectories for genuine and forged signatures
os.makedirs(os.path.join(train_dir, 'genuine'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'forged'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'genuine'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'forged'), exist_ok=True)

Splitting Data and Loading into Specific Directories

In [4]:
signature_dirs = [d for d in os.listdir('CEDAR') if os.path.isdir(os.path.join('CEDAR', d))]

# Split directories into train and val
random.seed(42)
random.shuffle(signature_dirs)
train_dirs = signature_dirs[:int(0.8 * len(signature_dirs))]
val_dirs = signature_dirs[int(0.8 * len(signature_dirs)):]

# Copy images to train and val directories
for d in train_dirs:
    dir_path = os.path.join('CEDAR', d)
    for img in os.listdir(dir_path):
        if img.startswith('original'):
            src = os.path.join(dir_path, img)
            dst = os.path.join(train_dir, 'genuine')
            shutil.copy(src, dst)
        elif img.startswith('forgeries'):
            src = os.path.join(dir_path, img)
            dst = os.path.join(train_dir, 'forged')
            shutil.copy(src, dst)

for d in val_dirs:
    dir_path = os.path.join('CEDAR', d)
    for img in os.listdir(dir_path):
        if img.startswith('original'):
            src = os.path.join(dir_path, img)
            dst = os.path.join(val_dir, 'genuine')
            shutil.copy(src, dst)
        elif img.startswith('forgeries'):
            src = os.path.join(dir_path, img)
            dst = os.path.join(val_dir, 'forged')
            shutil.copy(src, dst)

Loading Images

In [5]:

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 2112 images belonging to 2 classes.
Found 528 images belonging to 2 classes.


Defining CNN Architecture

In [16]:
# Model Architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

Training the Model

In [17]:
# Model Training
early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stop])

Epoch 1/50
66/66 [==============================] - 19s 255ms/step - loss: 0.6946 - accuracy: 0.5147 - val_loss: 0.6823 - val_accuracy: 0.7292
Epoch 2/50
66/66 [==============================] - 16s 245ms/step - loss: 0.6805 - accuracy: 0.5649 - val_loss: 0.6018 - val_accuracy: 0.7330
Epoch 3/50
66/66 [==============================] - 15s 229ms/step - loss: 0.6570 - accuracy: 0.6098 - val_loss: 0.6341 - val_accuracy: 0.6742
Epoch 4/50
66/66 [==============================] - 18s 271ms/step - loss: 0.6296 - accuracy: 0.6520 - val_loss: 0.5285 - val_accuracy: 0.7576
Epoch 5/50
66/66 [==============================] - 19s 283ms/step - loss: 0.6176 - accuracy: 0.6615 - val_loss: 0.4834 - val_accuracy: 0.7803
Epoch 6/50
66/66 [==============================] - 19s 289ms/step - loss: 0.5899 - accuracy: 0.6932 - val_loss: 0.4441 - val_accuracy: 0.8106
Epoch 7/50
66/66 [==============================] - 18s 276ms/step - loss: 0.5750 - accuracy: 0.7027 - val_loss: 0.3929 - val_accuracy: 0.8163

Evaluation Metrics

In [18]:
# Evaluate Model
val_loss, val_acc = model.evaluate(val_generator)
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_acc)

y_true = val_generator.classes
y_pred = (model.predict(val_generator) > 0.5).astype(int)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1)

17/17 [==============================] - 3s 137ms/step - loss: 0.2036 - accuracy: 0.9129
Validation Loss: 0.20361071825027466
Validation Accuracy: 0.9128788113594055
17/17 [==============================] - 3s 120ms/step
Precision: 0.481651376146789
Recall: 0.3977272727272727
F1-Score: 0.43568464730290457
